In [33]:
%load_ext autoreload
%matplotlib inline  
%autoreload 
__author__ = 'ewan'
import matplotlib as mpl
import sys
sys.path.append("../src/") 
import matplotlib.pyplot as plt
import numpy as np
import WindFarm as wf
import WindTurbine as wt
#import matplotlib.pyplot as plt
from DWM_flowfield_farm import DWM_main_field_model
from DWM_init_dict import init
from math import pi, sqrt, isnan
from DWM_GClarsenPicks import get_Rw_NOJ, gaussN, get_Rw, get_R96, Ua, Ua_shear
import time
from scipy import io, interpolate
from DWM_misc import smooth, to_bool
import matplotlib.pylab as plt
import matplotlib._cntr as cntr

###########################################################################
def sDWM(derating,**kwargs):
#     WD,WS,TI,WTcoord,WTG,HH,R,stab,accum,optim,
    WD = kwargs.get('WD')
    WS = kwargs.get('WS')
    TI = kwargs.get('TI') 
    WTcoord = kwargs.get('WTcoord') 
    WTG = kwargs.get('WTG') 
    HH = kwargs.get('HH') 
    R = kwargs.get('R') 
    stab = kwargs.get('stab') 
    accum = kwargs.get('accum') 
    optim = to_bool(kwargs.get('optim'))
    
#     print '************************** Run DWM model ******************'
#     print '***********************************************************'
#     print '***********************************************************'
    ttt = time.time()
    # WT = wt.WindTurbine('Vestas v80 2MW offshore','V80_2MW_offshore.dat',70,40)
    # WF = wf.WindFarm('Horns Rev 1','HR_coordinates.dat',WT)
    WT = wt.WindTurbine('Windturbine','../WT-data/'+WTG+'/'+WTG+'_PC.dat',HH,R)
    WF = wf.WindFarm('Windfarm',WTcoord,WT)
    if optim is True:
        print 'Performing optimization'
        WT.CP = np.load('../WT-data/'+WTG+'/'+WTG+'_CP.npy')
        WT.CT = np.load('../WT-data/'+WTG+'/'+WTG+'_CT.npy')
        WT.lambdaf3=WT.CP[:,:,0]
        WT.PITCH3=WT.CP[:,:,1]
        WT.CP3=WT.CP[:,:,2]
        WT.CT3=WT.CT[:,:,2]
        WT.CP3[WT.CP3>(16./27.)]=0
        WT.CP3[np.isnan(WT.CP3)]=0
        WT.CT3[np.isnan(WT.CT3)]=0
        WT.CPc = cntr.Cntr(WT.PITCH3,WT.lambdaf3,WT.CP3)
        WT.CTc=cntr.Cntr(WT.PITCH3,WT.lambdaf3,WT.CT3)
        # WT.CPcurve= np.load('WT-data/'+WTG+'/BEM/CPcurve.npy')
        # WT.CTcurve= np.load('WT-data/'+WTG+'/BEM/CTcurve.npy')
        # WT.getCTCP= interpolator(WT.CPcurve,WT.CTcurve)
    elif optim is False:
        print 'Not Performing optimization'
        derating=1.0*np.ones((WF.nWT))
        WT.CP = None
        WT.CT = None

    print 'Derating are', derating
    #WT = wt.WindTurbine('NREL 5 MW','NREL_5MW.dat',90,63.0)

    # Scaling wind farm to NREL's rotor size
    if 'Lill' in WTcoord:
        WF.vectWTtoWT=WF.vectWTtoWT*(WT.R/46.5) # 46.5 is the Lillgrund nominal radius of SWT turbine
    # Compute distance WT to WT in mean flow coordinate system
    distFlowCoord, nDownstream, id0= WF.turbineDistance(WD)

    # Init dictionnaries
    deficits, turb, inlets_ffor, inlets_ffor_deficits,inlets_ffor_turb,out, DWM, ID_waked, ID_wake_adj, Farm_p_out, WT_p_out, Vel_out,WT_pitch_out,WT_RPM_out=init(WF)

    # Extreme wake to define WT's in each wake, including partial wakes
    ID_wake = {id0[i]:(get_Rw(x=distFlowCoord[0,id0[i],:],\
                              R=2.*WF.WT.R,TI=TI,CT=WT.get_CT(WS),pars=[0.435449861,0.797853685,-0.124807893,0.136821858,15.6298,1.0])>\
               np.abs(distFlowCoord[1,id0[i],:])).nonzero()[0] \
               for i in range(WF.nWT)}
    # Power output list
    Farm_p_out=0.
    WT_p_out=np.zeros((WF.nWT))
    WT_pitch_out=np.zeros((WF.nWT,2))
    WT_RPM_out=np.zeros((WF.nWT,2))
    Vel_out=np.zeros((WF.nWT))
    # print ID_wake
    # fig,ax=WF.plot(WT_num=True)
    # fig.hold('True')
    # plt.show()

    for iT in range(WF.nWT):
        print 'Simulation WT: '+str(iT+1)+'/'+str(WF.nWT)
        # Define flow case geometry
        cWT = id0[iT]
        #Radial coordinates in cWT for wake affected WT's
        x=distFlowCoord[0,cWT,ID_wake[cWT]]
        C2C   = distFlowCoord[1,cWT,ID_wake[cWT]]

        index_orig=np.argsort(x)
        x=np.sort(x)
        row= ID_wake[id0[iT]][index_orig]
        C2C=C2C[index_orig]

        # Wrapping the DWM core model with I/O
        par={
         'WS':WS,
         'TI':TI,
         'atmo_stab':stab,
         'WTG':'NY2',      #'WTG':'NREL5MW',
         'WTG_spec': WT,
         'wtg_ind': row, # turbine index
         'hub_z':x/(2*WF.WT.R), # compute the flow field at downstream location of each downwind turbine !
         'hub_x': np.ceil((2*(max(abs(C2C))+WF.WT.R))/WF.WT.R)*0.5+C2C/(WF.WT.R), # lateral offset of each downwind turbine with respect to the most upstream turbine in the tree
         'C2C': C2C, # center 2 center distances between hubs
         'lx':np.ceil((2.*(max(abs(C2C))+WF.WT.R))/WF.WT.R), # length of the domain in lateral
         'ly':np.ceil((2.*(max(abs(C2C))+WF.WT.R))/WF.WT.R),  # length of the domain in longitudinal in D
         # 'wake_ind_setting':1,
         'accu_inlet': True,
         'derating': derating[row[0]],
         'optim': optim,
         'accu': accum, # type of wake accumulation
         'full_output': False # Flag for generating the complete output with velocity deficit
        }

        ID_wake_adj[str(id0[iT])]=row
        aero, meta, mfor, ffor, DWM, deficits,inlets_ffor,inlets_ffor_deficits, inlets_ffor_turb,turb, out,ID_waked = DWM_main_field_model(ID_waked,deficits,inlets_ffor,inlets_ffor_deficits,inlets_ffor_turb,turb,DWM,out,**par)
        # Total power
        Farm_p_out= Farm_p_out+out[str(meta.wtg_ind[0])][0] # based on BEM
        # Power by each turbine
        WT_p_out[iT]=out[str(meta.wtg_ind[0])][0]
        # Pitch and RPM
        WT_pitch_out[iT,0]=aero.PITCH
        WT_pitch_out[iT,1]=aero.PITCH_opt
        WT_RPM_out[iT,0]=aero.RPM
        WT_RPM_out[iT,1]=aero.RPM_opt
        Vel_out[iT]=out[str(meta.wtg_ind[0])][1]    
    elapsedttt = time.time() - ttt
    print 'Total run  for wind direction %i is %4.2f sec' %(WD,elapsedttt)
    # filename='outputs/dwm_WS_'+ str(WS) +'_dir_' + str(WD)+'_TI_' + str(TI)
    # filename.replace('.','_')
    # np.save(filename,out)
    # return aero, meta, mfor, ffor, DWM, deficits,inlets_ffor,inlets_ffor_deficits, turb, DWM, out, ID_waked, id0,ID_wake_adj
    print 'The farm production is: %4.2f kW, where each turbine is: %s' %(Farm_p_out,np.array_str(WT_p_out))
    # plt.close('all')
    return Farm_p_out,WT_p_out,ID_waked,ID_wake_adj


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
inputs=dict()
##########################################################################
inputs={'WD':120.0,
'WS':9.0,
'TI':0.06,
# WTcoord='Lill_coordinates.dat'
'WTcoord':'../data/Lill_A04B04C04.dat',
# 'WTcoord':'../data/Lill_full.dat',
#'WTcoord':'../data/Lill_row04.dat',      
'WTG':'NY2',
'HH':65.0,
'R':46.5,
'stab':'N',
'accum':'dominant',
'optim':True}
##########################################################################
WT = wt.WindTurbine('Windturbine','../WT-data/'+inputs['WTG']+'/'+inputs['WTG']+'_PC.dat',inputs['HH'],inputs['R'])
WF = wf.WindFarm('Windfarm',inputs['WTcoord'],WT)

Pnom=WT.get_P(inputs['WS']) # at 9m/s
# Pnom=1364.55
Pref=WF.nWT*WT.P_rated

Pref,Tref, ID_waked,ID_wake_adj= sDWM(np.ones((WF.nWT)),**inputs)

print 'Pnom is', Tref[0]
print 'PT are', Tref
print 'Deratings at max cp', Tref/WT.P_rated
# print 'Derating T2 at max cp', Tref[1]/WT.P_rated

dmaxT1=Tref[0]/WT.P_rated
dmaxT2=Tref[1]/WT.P_rated

vdmax=Tref/WT.P_rated

Performing optimization
Derating are [ 1.  1.  1.]
Simulation WT: 1/3
Simulation WT: 2/3
Simulation WT: 3/3
Total run  for wind direction 120 is 1.18 sec
The farm production is: 2128.21 kW, where each turbine is: [ 1386.81   417.75   323.65]
Pnom is 1386.81
PT are [ 1386.81   417.75   323.65]
Deratings at max cp [ 0.60294983  0.18162711  0.14071482]


In [35]:
# Baseline deratings 
# Here we try to define a relevant initial derating vector 
# The basic idea is to downregulated the most upstreams turbines
# Set no derating to the most downstream turbines
# In between allow variation +- around the baseline value
x0=vdmax*1.01*np.ones_like(vdmax)
m = max(vdmax)
upstreams=[i for i, j in enumerate(vdmax) if j == m]
x0[upstreams]=vdmax[upstreams]*0.96
no_downstream_turbine=np.asarray([len(ID_wake_adj[str(i)]) for i in range(WF.nWT)])
downstreams=[index for index,value in enumerate(no_downstream_turbine) if value <1.1]
print downstreams
x0[downstreams]=1.
print x0

[2]
[ 0.57883184  0.18344338  1.        ]


In [52]:
x0=[0.1,0.1,1.]
dmaxT1=max(x0)

In [53]:
from scipy.optimize import fmin_cobyla

Prated=WF.nWT*WT.P_rated
# dmaxT1=max(vdmax)

def objective(x,Prated,inputs):
#     derating=np.hstack((x, 1.))
    derating = x
    Ptot,Ttot,ID_waked,ID_wake_adj=sDWM(derating,**inputs) # power produced by the wind farm call to sDWM
    return abs(Ptot-Prated) # maximize power by derating

def constr1(x,dmax):
    return np.min(dmax*np.ones_like(x) - x)

def constr2(x,*args):
    return np.min(x - 0.05*np.ones_like(x))


# # x0=np.asarray([0.4,0.4,0.4,0.4,0.4,0.5,0.6,1.0])
# x0 = np.ones((WF.nWT-1.)) # init the init derating vector -1 cause last turbine
# # x0[0] = dmax*0.75 # init upstream derating of 25% ?

# # x0[0]=1.
# # x0[1]=1.
# x0[0]=dmaxT1*0.95
# x0[1]=dmaxT2*1.01
# # x0[2]=1.
# # x0[-1]= dmax


x=fmin_cobyla(objective , x0, cons=[constr1,constr2], args=(Pref,inputs), consargs=(dmaxT1,),rhobeg=0.1, rhoend=0.0025, iprint=1, maxfun=1000, disp=1, catol=0.002)
# x=fmin_cobyla(objective , x0, cons=[], args=(Pref,inputs),  consargs=None, rhobeg=0.01, rhoend=0.001, iprint=1, maxfun=1000, disp=1, catol=0.0002)

Performing optimization
Derating are [ 0.1  0.1  1. ]
Simulation WT: 1/3
Simulation WT: 2/3
Simulation WT: 3/3
Total run  for wind direction 120 is 5.72 sec
The farm production is: 1511.06 kW, where each turbine is: [  229.93   229.97  1051.16]
Performing optimization
Derating are [ 0.2  0.1  1. ]
Simulation WT: 1/3
Simulation WT: 2/3
Simulation WT: 3/3
Total run  for wind direction 120 is 6.08 sec
The farm production is: 1635.44 kW, where each turbine is: [ 459.88  229.94  945.62]
Performing optimization
Derating are [ 0.2  0.2  1. ]
Simulation WT: 1/3
Simulation WT: 2/3
Simulation WT: 3/3
Total run  for wind direction 120 is 7.12 sec
The farm production is: 1719.76 kW, where each turbine is: [ 459.88  460.15  799.73]
Performing optimization
Derating are [ 0.2  0.2  1.1]
Simulation WT: 1/3
Simulation WT: 2/3
Simulation WT: 3/3
Total run  for wind direction 120 is 7.19 sec
The farm production is: 1719.76 kW, where each turbine is: [ 459.88  460.15  799.73]
Performing optimization
Derat

In [54]:
print x
Popt,Topt,ID_waked, ID_wake_adj= sDWM(x,**inputs)

[ 0.51324153  0.3311613   0.95660231]
Performing optimization
Derating are [ 0.51324153  0.3311613   0.95660231]
Simulation WT: 1/3
Simulation WT: 2/3
Simulation WT: 3/3
Total run  for wind direction 120 is 4.16 sec
The farm production is: 2126.85 kW, where each turbine is: [ 1180.62   633.56   312.67]


In [55]:
Gainpower= 100*((Popt-Pref)/Popt)
print 'Derating T1 at max power', x[0]
print 'Derating T2 at max power', x[1]
print 'Derating T1 at max cp', Tref[0]/WT.P_rated
print 'Derating T2 at max cp', Tref[1]/WT.P_rated

print 'percentage of derating T1 at max power', 100*((Tref[0]/WT.P_rated-x[0])/(Tref[0]/WT.P_rated))
print 'percentage of derating T2 at max power', 100*((Tref[1]/WT.P_rated-x[1])/(Tref[1]/WT.P_rated))
print 'percentage of all derating at max power', 100*((Tref/WT.P_rated-x)/(Tref/WT.P_rated))
print 'power of all derating at max power', Pref
print 'power of all derating  at max cp', Popt
print 'turbines power variation',  100*((Topt-Tref)/Topt)
print 'Power yield gained from derating in %', Gainpower

Derating T1 at max power 0.513241526104
Derating T2 at max power 0.331161303891
Derating T1 at max cp 0.602949832962
Derating T2 at max cp 0.181627110217
percentage of derating T1 at max power 14.8782372851
percentage of derating T2 at max power -82.3303269518
percentage of all derating at max power [  14.87823729  -82.33032695 -579.81634051]
power of all derating at max power 2128.21
power of all derating  at max cp 2126.85
turbines power variation [-17.46455252  34.06307216  -3.51168964]
Power yield gained from derating in % -0.0639443308179


In [39]:
# # Canonic test
# from scipy.optimize import fmin_cobyla
# inputs=dict()
# ##########################################################################
# inputs={'WD':120.0,
# 'WS':9.0,
# 'TI':0.06,
# # WTcoord='Lill_coordinates.dat'
# 'WTcoord':'Lill_coordinates_canonic.dat',
# 'WTG':'NY2',
# 'HH':65.0,
# 'R':46.5,
# 'stab':'N',
# 'accum':'dominant',
# 'optim':True}
# ##########################################################################
# WT = wt.WindTurbine('Windturbine','WT-data/'+inputs['WTG']+'/'+inputs['WTG']+'_PC.dat',inputs['HH'],inputs['R'])
# WF = wf.WindFarm('Windfarm',inputs['WTcoord'],WT)

# Pnom=WT.get_P(inputs['WS']) # at 9m/s
# # Pnom=1364.55
# Pref=WF.nWT*WT.P_rated
# dmax=Pnom/WT.P_rated
# print 'Pnom is', Pnom
# print 'dmax is', dmax
# # Ptots=np.zeros((len(np.arange(0.06,0.76,0.1)),len(np.arange(0.06,0.76,0.1)),len(np.arange(0.06,0.76,0.1))))

# vT1=np.linspace(0.90*dmax,dmax,25)
# vT2=np.linspace(0.010,0.19,25)

# Ptots=np.zeros((len(vT1),len(vT2)))
# Trefs=np.zeros( (  WF.nWT,   len(vT1),len(vT2 )))
# print Ptots.shape
# k1=0
# for iT1 in vT1:
#     k2=0
#     for iT2 in vT2:
# #         k3=0
# #         for iT3 in np.arange(0.06,0.76,0.1):
#         x=[iT1,iT2,1.]
#         Ptot,Tref=sDWM(x,**inputs)
# #         Ptots[k1,k2,k3]=Ptot
#         Ptots[k1,k2]=Ptot
#         Trefs[:,k1,k2]=Tref
# #             k3=k3+1
#         k2=k2+1
#     k1=k1+1
            
# #             print Ptots

In [40]:
# save lengthy for loop computations
# np.save('Ptots3wtt.npy',Ptots)
# np.save('Trefs3wtt.npy',Trefs)

In [41]:
# import numpy as np
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import cm
# from matplotlib.ticker import LinearLocator, FormatStrFormatter
# import matplotlib.pyplot as plt
# %matplotlib inline  

# # Ptots=np.load('Ptots.npy')
# Ptots=np.load('Ptots3wtt.npy')
# Trefs=np.load('Trefs3wtt.npy')
# ##########################################################################
# inputs={'WD':120.0,
# 'WS':9.0,
# 'TI':0.06,
# # WTcoord='Lill_coordinates.dat'
# 'WTcoord':'Lill_coordinates_canonic.dat',
# 'WTG':'NY2',
# 'HH':65.0,
# 'R':46.5,
# 'stab':'N',
# 'accum':'dominant',
# 'optim':True}
# ##########################################################################
# WT = wt.WindTurbine('Windturbine','WT-data/'+inputs['WTG']+'/'+inputs['WTG']+'_PC.dat',inputs['HH'],inputs['R'])
# WF = wf.WindFarm('Windfarm',inputs['WTcoord'],WT)

# Pnom=WT.get_P(inputs['WS']) # at 9m/s
# Pref,Tref= sDWM([1.0,1.0,1.0],**inputs)
# # Pnom=1364.55
# dmax=Pnom/WT.P_rated

# # print 'dmax is', dmax

# dT1=np.asarray([])
# dT2=np.asarray([])
# # dT3=np.asarray([])
# for iT1 in vT1:
#     for iT2 in vT2:
# #         for iT3 in np.arange(0.06,0.71,0.05):
#         dT1=np.append(dT1,iT1)
#         dT2=np.append(dT2,iT2)
# #             dT3=np.append(dT3,iT3)

# Ptotsf=Ptots.flatten()
# Trefsf=Trefs.flatten()

# # print '2d index', np.asarray(np.unravel_index(Ptots.argmax(), Ptots.shape))
# opt_index= np.asarray(np.unravel_index(Ptots.argmax(), Ptots.shape))
# print opt_index
# Gainpower= 100*((max(Ptotsf)-Pref)/max(Ptotsf))
# # print Trefs[:,opt_index[0],opt_index[1]]

# print vT2

# # print dT1.shape
# print 'Maximum power from optimization %4.2f/ max CP power %4.2f' %(max(Ptotsf) , Pref)
# # print 'index', np.argmax(Ptotsf)
# print 'Derating T1 at max power', dT1[np.argmax(Ptotsf)]
# print 'Derating T2 at max power', dT2[np.argmax(Ptotsf)]
# print 'Derating T1 at max cp', Tref[0]/WT.P_rated
# print 'Derating T2 at max cp', Tref[1]/WT.P_rated
# print 'percentage of derating T1 at max power', 100*((Tref[0]/WT.P_rated-dT1[np.argmax(Ptotsf)])/(Tref[0]/WT.P_rated))
# print 'percentage of derating T2 at max power', 100*((Tref[1]/WT.P_rated-dT2[np.argmax(Ptotsf)])/(Tref[1]/WT.P_rated))
# print 'Power yield gained from derating in %', Gainpower
# print ' .... Power produced by 3 turbines at standard', Tref
# print ' .... .Optimized power producted by 3 turbines', Trefs[:,opt_index[0],opt_index[1]]


# plt.figure(figsize=(15,5))
# plt.subplot(1,2,1)
# # plt.contourf( np.arange(0.05,dmax+0.075,0.0125),    np.arange(0.05,dmax+0.075,0.0125),100.*((Ptots.T-Pref)/Ptots.T),150, cmap=cm.seismic,vmin=0,vmax=3)
# plt.contourf( vT1,    vT2,Ptots.T,150, cmap=cm.seismic)
# plt.plot(dT1[np.argmax(Ptotsf)],dT2[np.argmax(Ptotsf)],'wo',markersize=15)
# plt.plot(Tref[0]/WT.P_rated,Tref[1]/WT.P_rated,'ko',markersize=15)
# plt.xlabel('Derating T1 [-]')
# plt.ylabel('Derating T2 [-]')
# plt.colorbar()


# plt.subplot(1,2,2)
# # plt.contourf( np.arange(0.05,dmax+0.075,0.0125),    np.arange(0.05,dmax+0.075,0.0125),100.*((Ptots.T-Pref)/Ptots.T),150, cmap=cm.seismic,vmin=0,vmax=3)
# plt.contourf( vT1,    vT2,100.*((Ptots.T-Pref)/Ptots.T),150, cmap=cm.seismic)
# plt.plot(dT1[np.argmax(Ptotsf)],dT2[np.argmax(Ptotsf)],'wo',markersize=15)
# plt.plot(Tref[0]/WT.P_rated,Tref[1]/WT.P_rated,'ko',markersize=15)
# plt.xlabel('Derating T1 [-]')
# plt.ylabel('Derating T2 [-]')
# plt.colorbar()



# # plt.figure(figsize=(15,5))
# # plt.subplot(1,2,1)
# # # plt.contourf( np.arange(0.05,dmax+0.075,0.0125),    np.arange(0.05,dmax+0.075,0.0125),100.*((Ptots.T-Pref)/Ptots.T),150, cmap=cm.seismic,vmin=0,vmax=3)
# # plt.contourf( 100*((Tref[0]/WT.P_rated-vT1)/(Tref[0]/WT.P_rated)),   100*((Tref[1]/WT.P_rated-vT2)/(Tref[1]/WT.P_rated)),Ptots.T,150, cmap=cm.hsv)
# # plt.plot(100*(Tref[0]/WT.P_rated-dT1[np.argmax(Ptotsf)])/(Tref[0]/WT.P_rated),100*(Tref[1]/WT.P_rated-dT2[np.argmax(Ptotsf)])/(Tref[1]/WT.P_rated),'wo',markersize=10)
# # plt.plot(0,0,'w+',markersize=10)
# # plt.xlabel('Derating T1 [-]')
# # plt.ylabel('Derating T2 [-]')
# # plt.colorbar()


# # plt.subplot(1,2,2)
# # # plt.contourf( np.arange(0.05,dmax+0.075,0.0125),    np.arange(0.05,dmax+0.075,0.0125),100.*((Ptots.T-Pref)/Ptots.T),150, cmap=cm.seismic,vmin=0,vmax=3)
# # plt.contourf( 100*((Tref[0]/WT.P_rated-vT1)/(Tref[0]/WT.P_rated)),   100*((Tref[1]/WT.P_rated-vT2)/(Tref[1]/WT.P_rated)),100.*((Ptots.T-Pref)/Ptots.T),150, cmap=cm.hsv,vmin=0,vmax=3)
# # plt.plot(100*(Tref[0]/WT.P_rated-dT1[np.argmax(Ptotsf)])/(Tref[0]/WT.P_rated),100*(Tref[1]/WT.P_rated-dT2[np.argmax(Ptotsf)])/(Tref[1]/WT.P_rated),'wo',markersize=10)
# # plt.plot(0,0,'w+',markersize=10)
# # plt.xlabel('Derating T1 [-]')
# # plt.ylabel('Derating T2 [-]')
# # plt.colorbar()


# plt.figure()
# plt.plot(100*((Tref[0]/WT.P_rated-vT1)/(Tref[0]/WT.P_rated)),Ptots[: ,opt_index[1]]/Pref,'ko--')
# plt.hold(True)
# # plt.plot()
# plt.grid('on')
# plt.xlabel('Derating T1 [-]')
# plt.ylabel('Power of 3 turbines [kW]')

# plt.figure()
# plt.plot(100*((Tref[1]/WT.P_rated-vT2)/(Tref[1]/WT.P_rated)),Ptots[opt_index[0] ,:]/Pref,'ko--')
# plt.hold(True)
# # plt.plot()
# plt.grid('on')
# plt.xlabel('Derating T2 [-]')
# plt.ylabel('Power of 3 turbines [kW]')

In [42]:
# fig = plt.figure(figsize=(10, 5))
# ax = Axes3D(fig)
# X,Y=np.meshgrid( vT1, vT2)
# surf=ax.plot_surface( X,Y,Ptots, rstride=1, cstride=1, cmap=cm.seismic,
#         linewidth=0, antialiased=False,shade=True,vmin=1200,vmax=2150,alpha=0.5)
# ax.scatter(dT2[np.argmax(Ptotsf)],dT1[np.argmax(Ptotsf)],max(Ptotsf), marker='o',s=200)
# # ax.set_zlim(0., 0.5)

# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# fig.colorbar(surf, shrink=0.5, aspect=5)

In [43]:
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
# %matplotlib inline  

# # Plots
# Ptotsf=np.load('outputs/Optim/Ptotsf.npy')
# Ptots=np.load('outputs/Optim/Ptots.npy')
# dT1=np.load('outputs/Optim/dT1.npy')
# dT2=np.load('outputs/Optim/dT2.npy')
# dT3=np.load('outputs/Optim/dT3.npy')

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d') 
# sc=ax.scatter(dT1, dT2, dT3, c=Ptotsf, cmap=plt.hsv())
# plt.colorbar(sc)
# plt.show() 

# print max(Ptotsf)
# print np.argmax(Ptotsf)
# print dT1[np.argmax(Ptotsf)]
# print dT2[np.argmax(Ptotsf)]
# print dT3[np.argmax(Ptotsf)]

In [44]:
# inputs=dict()
# ##########################################################################
# inputs={'WD':120.0,
# 'WS':9.0,
# 'TI':0.06,
# # WTcoord='Lill_coordinates.dat'
# 'WTcoord':'Lill_coordinates_canonic.dat',
# 'WTG':'NY2',
# 'HH':65.0,
# 'R':46.5,
# 'stab':'N',
# 'accum':'dominant',
# 'optim':True}
# ##########################################################################
# Pref= sDWM([1.0,1.0,1.0],**inputs)

# # Gainpower= 100*((max(Ptotsf)-Pref)/max(Ptotsf))
# print 'Power gain from derating in %', Gainpower
# print 'Amount of derating T1', 1-(0.525/dmax)
# print 'Amount of derating T2', 1-(0.1625/dmax)
# Poptmax= sDWM([0.525,0.1825,1.0],**inputs)

In [45]:
# 0.51*2300

In [46]:
# plt.figure()
# plt.plot(Ptotsf)
# plt.plot(np.argmax(Ptotsf),Ptotsf[np.argmax(Ptotsf)],'ro')

In [47]:
# # Canonic for 3D scalar plot
# import numpy
# from mayavi.mlab import *

# def test_contour3d(scalars):
#     x, y, z = numpy.ogrid[0.06:0.05:0.71j, 0.06:0.05:0.71j, 0.06:0.05:0.71j]
# #     scalars = x * x * 0.5 + y * y + z * z * 2.0

#     obj = contour3d(scalars, contours=4, transparent=True)
#     return obj